[View in Colaboratory](https://colab.research.google.com/github/juan-bolivar/BTC_Trading/blob/master/BTC_TRADING.ipynb)

# Nueva sección

In [2]:
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import pdb

EPISODES = 5000


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state in minibatch:
            
            target = self.model.predict(state)
            t = self.target_model.predict(next_state)[0]
            target[0][action] = reward + self.gamma * np.amax(t)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)




Using TensorFlow backend.


In [0]:


"""Analyze a portfolio.

Copyright 2017, Georgia Tech Research Corporation
Atlanta, Georgia 30332-0415
All Rights Reserved
"""

import pandas as pd
import numpy as np
import datetime as dt
#from util import get_data, plot_data
import matplotlib.pyplot as plt
import pdb

# This is the function that will be tested by the autograder
# The student must update this code to properly implement the functionality


def daily_returns(df):
    df1 = df.copy()
    df2 = (df1[1:]/df1[:-1].values) - 1
    return(df2)

def compound_returns(df):
    df1 = df.copy()
    df1 = df1/df1.iloc[0] - 1
    df1.iloc[0,] = 0 
    return(df1)
def normalize_data(df):
    df1 = df.copy()
    return (df1/df1.iloc[0])
    
def plot_data(df,title='Normalized Data'):
    ax=df.plot(title=title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.grid(linestyle='--', linewidth=1)
    plt.show()
    


def assess_portfolio(portfolio=None,sd = dt.datetime(2008,1,1), ed = dt.datetime(2009,1,1), \
    syms = ['GOOG','AAPL','GLD','XOM'], \
    allocs=[0.1,0.2,0.3,0.4], \
    sv=1000000, rfr=0.0, sf=252.0,precios=0, \
    gen_plot=False):

    # Read in adjusted closing prices for given symbols, date range
    dates = pd.date_range(sd, ed)
    if(type(portfolio)==type(None)):
        prices_all = get_data(syms, dates)  # automatically adds SPY
        prices = prices_all[syms]  # only portfolio symbols
        prices_SPY = prices_all['SPY']  # only SPY, for comparison later
    else:
        #pdb.set_trace()
        prices_all = get_data(syms,dates)
        prices = portfolio# pd.concat([portfolio,prices_all[syms]],axis=1)
        prices_SPY = prices_all[syms]
    # Get daily portfolio value

    
    
    
    if( (isinstance(precios,pd.core.series.Series))):
        pdb.set_trace()
        precios = precios.to_frame()
        normalized_values = normalize_data(precios)
        
    else:
        normalized_values = normalize_data(prices)

    
    alloced_values    = normalized_values*allocs
    pos_val           = alloced_values*sv
    port_val          = pos_val.sum(axis=1) # add code here to compute daily portfolio values

    d_returns         = daily_returns(port_val)
    cr                = port_val.iloc[-1]/port_val.iloc[0] -1
    adr               = d_returns.mean()
    sddr              = d_returns.std()
    sr                = (adr/sddr)*np.sqrt(252)
    prices_SPY = normalize_data(prices_SPY)
    port_val   = normalize_data(port_val)
    port_val   = pd.DataFrame(data=port_val,index=port_val.index,columns=['Portfolio'])
    # Get portfolio statistics (note: std_daily_ret = volatility)
    #cr, adr, sddr, sr = [0.25, 0.001, 0.0005, 2.1] # add code here to compute stats

    # Compare daily portfolio value with SPY using a normalized plot
    if gen_plot:
        # add code to plot here
        if(type(portfolio)==type(None)):
            df_temp = pd.concat([port_val, prices_SPY], keys=['Portfolio', 'SPY'], axis=1)
        else:
            #pdb.set_trace()
            df_temp = pd.concat([port_val, prices_SPY], axis=1)
        plot_data(df_temp)

    # Add code here to properly compute end value
    ev = sv

    return(cr, adr, sddr, sr, ev)






  


In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import pdb
import matplotlib.pyplot as plt


def daily_returns(df):
    df1 = df.copy()
    df2 = (df1[1:]/df1[:-1].values) - 1
    return(df2)

def compound_returns(df):
    df1 = df.copy()
    df1 = df1/df1.iloc[0] - 1
    df1.iloc[0,] = 0 
    return(df1)

def normalize_data(df):
    df1 = df.copy()
    
    return (df1/df1.iloc[0])


def momentumfun(prices,n=14):

    salida = np.zeros_like(prices)
    salida = (prices.shift(-1)/prices)-1
    salida.iloc[:n,:]=np.nan
    return salida



def rsifunction(prices,n=14):
    #pdb.set_trace()
    deltas = prices.diff()
    up_rets = deltas[deltas>=0].fillna(0).cumsum()
    down_rets =-1*deltas[deltas<0].fillna(0).cumsum()
    
    
    up_gain = prices.copy()
    up_gain.ix[:] = 0
    up_gain.values[n:,:] = up_rets.values[n:,:] - up_rets.values[:-n,:]
    
    down_loss = prices.copy()
    down_loss.ix[:] = 0
    down_loss.values[n:,:] = down_rets.values[n:,:] - down_rets.values[:-n,:]
    
    #seed = deltas[:n+1]
    #up = seed[seed>=0].sum()/n
    #down = seed[seed<0].sum()/n
    #rs = up/down
    #rsi = np.zeros_like(prices)
    #rsi[:n] = 100. - 100./(1.+rs)

    rsi = prices.copy()
    rsi.ix[:,:]=0

    rs = (up_gain/n)/(down_loss/n)
    rsi = 100-(100/(1+rs))
    
    #for day in range(len(prices)):
    #   up            = up_gain.ix[day,:]
    #    down          = down_loss.ix[day,:]
    #    rs            = (up/n)/(down/n)
    #    rsi.ix[day,:] = 100-(100/(1+rs))

    rsi[rsi==np.inf]=100
    rsi.ix[:n,:]=np.nan
        
    
    
    
    #for i in range(n,len(prices)):
    #    delta = deltas.iloc[i-1]
    #    if delta > 0:
    #        upval = delta
    #        downval = 0
#
#        else:
#            upval = 0
#            downval = - delta
#        
#        up = (up*(n-1)+upval)/n
#        down = (down*(n-1)+downval)/n
#
#        rs = up/down
#        
#        rsi[i] = 100. - 100./(1.+rs)

    #rsi = pd.DataFrame(data=rsi , index = prices.index)
    #rsi.iloc[:n] = np.nan
    return rsi
                   
    
def plot_data(df,title='Normalized Data'):

    #fig, ax = plt.subplots()

    df1 =df['upp_std']
    
    plt.subplot(311)
    plt.plot('upp_std',data=df,linewidth=2,linestyle='--')
    plt.plot('down_std',data=df,linewidth=2,linestyle='--')
    plt.plot('moving_avarage',data=df,linewidth=2)
    plt.plot('Portfolio',data=df,linewidth=1)
    plt.subplot(312)
    plt.plot('rsi_val',data=df,linewidth=2)
    plt.subplot(313)
    plt.plot('momentum',data=df,linewidth=2)
    plt.show()

def indicators(sd = dt.datetime(2008,1,1), ed = dt.datetime(2009,12,31), \
    syms = ['JPM'], \
    allocs=[1], \
    sv=1000000, rfr=0.0, sf=252.0,precios=0, \
    gen_plot=False):
    #pdb.set_trace()
    # Read in adjusted closing prices for given symbols, date range
    dates = pd.date_range(sd, ed)
    prices_all = get_data(syms, dates)  # automatically adds SPY
    prices = prices_all[syms]  # only portfolio symbols
    prices_SPY = prices_all['SPY']  # only SPY, for comparison later
    # Get daily portfolio value

   
    normalized_values = normalize_data(prices)
   
    alloced_values    = normalized_values*allocs
    pos_val           = alloced_values*sv
    port_val          = pos_val.sum(axis=1) # add code here to compute daily portfolio values
    d_returns         = daily_returns(port_val)
   
    prices_SPY = normalize_data(prices_SPY)
    port_val   = normalize_data(port_val)

    moving_avarage    = normalized_values.rolling(14,min_periods=None,freq=None,center=False).mean()
    rolling_std       = normalized_values.rolling(14,min_periods=None,freq=None,center=False).std()

    rsi_val           = rsifunction(normalized_values)
    upp_std           = moving_avarage + 2*rolling_std
    down_std          = moving_avarage - 2*rolling_std
    momentum          = momentumfun(normalized_values)
    moving_avarage    = normalized_values/moving_avarage
    bbp               = (normalized_values-down_std)/(upp_std-down_std)
    sma_cross         = pd.DataFrame(0,columns=moving_avarage.columns,index=moving_avarage.index)
    sma_cross[moving_avarage>=1] =1
    sma_cross[1:] = sma_cross.diff()
    sma_cross.ix[0] = 0
    rsi_spy = rsifunction(normalize_data(prices_all))
    rsi_spy = pd.DataFrame(data=rsi_spy['SPY'],index=rsi_spy.index)
   
   
   
   
    # Get portfolio statistics (note: std*_daily_ret = volatility)
    #cr, adr, sddr, sr = [0.25, 0.001, 0.0005, 2.1] # add code here to compute stats

    # Compare daily portfolio value with SPY using a normalized plot
    if gen_plot:
        # add code to plot here
        df_temp = pd.concat([port_val, prices_SPY,moving_avarage,upp_std,down_std,rsi_val,momentum], keys=['Portfolio','SPY','moving_avarage','upp_std','down_std','rsi_val','momentum'], axis=1)
        plot_data(df_temp)
       
    return (normalized_values ,bbp,moving_avarage,rsi_val,rsi_spy,momentum,sma_cross)
#indicators(gen_plot=True)
#indicators(gen_plot=True)

print('hola mundo')


hola mundo


In [5]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import pdb
import matplotlib.pyplot as plt


def daily_returns(df):
    df1 = df.copy()
    df2 = (df1[1:]/df1[:-1].values) - 1
    return(df2)

def compound_returns(df):
    df1 = df.copy()
    df1 = df1/df1.iloc[0] - 1
    df1.iloc[0,] = 0 
    return(df1)

def normalize_data(df):
    df1 = df.copy()
    
    return (df1/df1.iloc[0])


def momentumfun(prices,n=14):

    salida = np.zeros_like(prices)
    salida = (prices.shift(-1)/prices)-1
    salida.iloc[:n,:]=np.nan
    return salida



def rsifunction(prices,n=14):
    #pdb.set_trace()
    deltas = prices.diff()
    up_rets = deltas[deltas>=0].fillna(0).cumsum()
    down_rets =-1*deltas[deltas<0].fillna(0).cumsum()
    
    
    up_gain = prices.copy()
    up_gain.ix[:] = 0
    up_gain.values[n:,:] = up_rets.values[n:,:] - up_rets.values[:-n,:]
    
    down_loss = prices.copy()
    down_loss.ix[:] = 0
    down_loss.values[n:,:] = down_rets.values[n:,:] - down_rets.values[:-n,:]
    
    #seed = deltas[:n+1]
    #up = seed[seed>=0].sum()/n
    #down = seed[seed<0].sum()/n
    #rs = up/down
    #rsi = np.zeros_like(prices)
    #rsi[:n] = 100. - 100./(1.+rs)

    rsi = prices.copy()
    rsi.ix[:,:]=0

    rs = (up_gain/n)/(down_loss/n)
    rsi = 100-(100/(1+rs))
    
    #for day in range(len(prices)):
    #   up            = up_gain.ix[day,:]
    #    down          = down_loss.ix[day,:]
    #    rs            = (up/n)/(down/n)
    #    rsi.ix[day,:] = 100-(100/(1+rs))

    rsi[rsi==np.inf]=100
    rsi.ix[:n,:]=np.nan
        
    
    
    
    #for i in range(n,len(prices)):
    #    delta = deltas.iloc[i-1]
    #    if delta > 0:
    #        upval = delta
    #        downval = 0
#
#        else:
#            upval = 0
#            downval = - delta
#        
#        up = (up*(n-1)+upval)/n
#        down = (down*(n-1)+downval)/n
#
#        rs = up/down
#        
#        rsi[i] = 100. - 100./(1.+rs)

    #rsi = pd.DataFrame(data=rsi , index = prices.index)
    #rsi.iloc[:n] = np.nan
    return rsi
                   
    
def plot_data(df,title='Normalized Data'):

    #fig, ax = plt.subplots()

    df1 =df['upp_std']
    
    plt.subplot(311)
    plt.plot('upp_std',data=df,linewidth=2,linestyle='--')
    plt.plot('down_std',data=df,linewidth=2,linestyle='--')
    plt.plot('moving_avarage',data=df,linewidth=2)
    plt.plot('Portfolio',data=df,linewidth=1)
    plt.subplot(312)
    plt.plot('rsi_val',data=df,linewidth=2)
    plt.subplot(313)
    plt.plot('momentum',data=df,linewidth=2)
    plt.show()

def indicators(allocs=[1],sv=1000000, rfr=0.0, sf=252.0,precios=0,gen_plot=False,data=pd.DataFrame()):
    #pdb.set_trace()
    # Read in adjusted closing prices for given symbols, date range
    #dates = pd.date_range(sd, ed)
    #prices_all = get_data(syms, dates)  # automatically adds SPY
    #prices = prices_all[syms]  # only portfolio symbols
    prices = data['Weighted_Price'].to_frame() #prices_all['SPY']  # only SPY, for comparison later
    # Get daily portfolio value

    
    normalized_values = normalize_data(prices)
    
    alloced_values    = normalized_values*allocs
    pos_val           = alloced_values*sv
    port_val          = pos_val.sum(axis=1) # add code here to compute daily portfolio values
    d_returns         = daily_returns(port_val)
   
    #prices_SPY = normalize_data(prices_SPY)
    port_val   = normalize_data(port_val)

    moving_avarage    = normalized_values.rolling(14,min_periods=None,freq=None,center=False).mean()
    rolling_std       = normalized_values.rolling(14,min_periods=None,freq=None,center=False).std()

    rsi_val           = rsifunction(normalized_values)
    upp_std           = moving_avarage + 2*rolling_std
    down_std          = moving_avarage - 2*rolling_std
    momentum          = momentumfun(normalized_values)
    moving_avarage    = normalized_values/moving_avarage
    bbp               = (normalized_values-down_std)/(upp_std-down_std)
    sma_cross         = pd.DataFrame(0,columns=moving_avarage.columns,index=moving_avarage.index)
    sma_cross[moving_avarage>=1] =1
    sma_cross[1:] = sma_cross.diff()
    sma_cross.ix[0] = 0
    #rsi_spy = rsifunction(normalize_data(prices_all))
    #rsi_spy = pd.DataFrame(data=rsi_spy['SPY'],index=rsi_spy.index)
    
    
    
    
    # Get portfolio statistics (note: std*_daily_ret = volatility)
    #cr, adr, sddr, sr = [0.25, 0.001, 0.0005, 2.1] # add code here to compute stats

    # Compare daily portfolio value with SPY using a normalized plot
    if gen_plot:
        # add code to plot here
        df_temp = pd.concat([port_val,moving_avarage,upp_std,down_std,rsi_val,momentum], keys=['Portfolio','moving_avarage','upp_std','down_std','rsi_val','momentum'], axis=1)
        plot_data(df_temp)
        
    return (normalized_values ,bbp,moving_avarage,rsi_val,momentum,sma_cross)
#indicators(gen_plot=True)

print('hola mundo')


hola mundo


In [0]:
import pdb 
import pandas as pd
import numpy as np
import datetime as dt
import os
import pdb

def author():
    return 'juan'

def compute_portvals(orders_df , start_val = 1000000, commission=9.95, impact=0.005):
    
    df_trades2 = orders_df.copy()
    df_trades2.sort_values('Date',inplace=True)
    df_trades2.loc[df_trades2['Order']=='BUY','Order']  = 1.0
    df_trades2.loc[df_trades2['Order']=='SELL','Order'] = -1.0
    df_trades2['Order'] = pd.to_numeric(df_trades2['Order'])
    
    df_trades2['Shares']=df_trades2['Shares']*df_trades2['Order']
    symbols = list(set(df_trades2.Symbol))
    df_trades = pd.DataFrame(columns=['Date']+symbols)
    inicio = 0

    for key,row in df_trades2.iterrows():
        df_trades.loc[inicio,row['Symbol']] = row['Shares']
        df_trades.loc[inicio,'Date'] = row['Date']
        inicio = inicio + 1
    
    df_trades = df_trades.fillna(0)

    start_date = df_trades.ix[0,'Date']
    end_date   = df_trades['Date'].iloc[-1]

    
    df_prices = get_data(symbols, pd.date_range(start_date, end_date))
    df_prices = df_prices[symbols]
    
    temp = pd.DataFrame(index=df_prices.index)
    df_trades = temp.join(df_trades.set_index('Date'),sort='True').rename_axis(None,axis=0).fillna(0)
    
    df_holdings = df_trades.copy()
    
    rv = pd.DataFrame(index=df_prices.index, data=df_prices.as_matrix(),columns=symbols)
    
    df_trades.index = pd.to_datetime(df_trades.index)
    df_trades = df_trades.mul(rv,axis='columns').dropna().fillna(0)
    df_diff = pd.DataFrame(index=df_trades.index)
    df_diff['impact']    = (abs(df_trades)*impact).sum(axis=1)
    df_diff['commission'] = (df_trades.iloc[:,:]!=0).sum(axis=1)*commission
    
    df_trades['Cash'] = -1*df_trades.sum(axis=1) - df_diff['commission'] - df_diff['impact']

    df_holdings = df_holdings.cumsum(axis=0)


    df_holdings['Cash'] =  start_val + df_trades['Cash'].cumsum(axis=0)

    temp           = df_holdings.iloc[:,:-1].copy()
    df_value       = temp.mul(df_prices,axis='columns')
    df_value['Value'] = df_value.sum(axis=1)

    df_portval = pd.DataFrame(index=list(range(len(df_value.index.get_values()))))
    #pdb.set_trace()
    df_portval['TotalValue']=0
    posicion = 0
    for key,value in df_value.iterrows():
        df_portval.ix[posicion,'TotalValue'] = df_value.ix[posicion,'Value'] + df_holdings.ix[posicion,'Cash']
        posicion = posicion + 1

    df_portval['Dates'] = df_value.index.get_values()
    
    df_portval = df_portval.drop_duplicates(subset='Dates',keep='last')
   
    df_portval = pd.DataFrame(data=df_portval.TotalValue.values,index=df_portval.Dates.values,columns=['TotalValue'])
    
    pdb.set_trace()
    #pd.concat([df_value['Value'],df_holdings['Cash'],df_portval,orders_df['Order']],axis=1)
    return df_portval


def test_code():
    

    #of = "./orders/orders2.csv"
    #df_trades = pd.read_csv(of,sep=',')
    pdb.set_trace()
    of  = "./salida.csv"
    df_trades  = pd.read_csv(of)

    sv = 1000000
    
    portvals = compute_portvals(orders_df=df_trades, start_val = sv,impact=0.005,commission=9.95)

    if isinstance(portvals, pd.DataFrame):
        portvals = portvals[portvals.columns[0]] 
    else:
        "warning, code did not return a DataFrame"

    start_date = portvals.index[0]
    end_date = portvals.index[-1]
    
    cum_ret, avg_daily_ret, std_daily_ret, sharpe_ratio,ev = assess_portfolio(sd=start_date,ed=end_date,allocs=[1],sv=sv,precios=portvals)
    cum_ret_SPY, avg_daily_ret_SPY, std_daily_ret_SPY, sharpe_ratio_SPY,ev = assess_portfolio(sd=start_date,ed=end_date,allocs=[1],sv=sv,syms=['SPY'])
    

    
    print("Date Range: {} to {}".format(start_date, end_date))
    print()
    print("Sharpe Ratio of Fund: {}".format(sharpe_ratio))
    print("Sharpe Ratio of SPY : {}".format(sharpe_ratio_SPY))
    print()
    print("Cumulative Return of Fund: {}".format(cum_ret))
    print("Cumulative Return of SPY : {}".format(cum_ret_SPY))
    print() 
    print("Standard Deviation of Fund: {}".format(std_daily_ret))
    print("Standard Deviation of SPY : {}".format(std_daily_ret_SPY))
    print()
    print("Average Daily Return of Fund: {}".format(avg_daily_ret))
    print("Average Daily Return of SPY : {}".format(avg_daily_ret_SPY))
    print()
    print("Final Portfolio Value: {}".format(portvals[-1]))


import pdb 
import pandas as pd
import numpy as np
import datetime as dt
import os
import pdb

def author():
    return 'juan'


def get_data(archive='COINBASE_FILTERED.csv',start_date=dt.datetime(2009,1,1) , end_date=dt.datetime(2010,1,1)):
    
    datos = pd.read_csv('https://www.dropbox.com/s/rmh9rramnsya81h/COINBASE_FILTERED.CSV?dl=1',sep=',')
    datos.index = pd.to_datetime(datos['Timestamp'],infer_datetime_format =True,unit='s')
    salida= datos.loc[start_date:end_date]
    return salida['Weighted_Price']
    
    
    
    

def compute_portvals(orders_df , start_val = 1000000, commission=9.95, impact=0.005):
    
    df_trades2 = orders_df.copy()
    df_trades2.sort_values('Date',inplace=True)
    df_trades2.loc[df_trades2['Order']=='BUY','Order']  = 1
    df_trades2.loc[df_trades2['Order']=='SELL','Order'] = -1
    df_trades2['Shares']=df_trades2['Shares']*df_trades2['Order']
    symbols = list(set(df_trades2.Symbol))
    df_trades = pd.DataFrame(columns=['Date']+symbols)
    inicio = 0

    for key,row in df_trades2.iterrows():
        df_trades.loc[inicio,row['Symbol']] = row['Shares']
        df_trades.loc[inicio,'Date'] = row['Date']
        inicio = inicio + 1
    
    df_trades = df_trades.fillna(0)

    start_date = df_trades.ix[0,'Date']
    end_date   = df_trades['Date'].iloc[-1]

    pdb.set_trace()
    
    df_prices = get_data('https://www.dropbox.com/s/rmh9rramnsya81h/COINBASE_FILTERED.CSV?dl=1',start_date=start_date,end_date=end_date)
    #df_prices = df_prices[symbols]
    
    temp = pd.DataFrame(index=df_prices.index)
    
    df_trades = temp.join(df_trades.set_index('Date'),sort='True').rename_axis(None,axis=0).fillna(0)
    
    df_holdings = df_trades.copy()
    
    rv = pd.DataFrame(index=df_prices.index, data=df_prices.as_matrix(),columns=symbols)
    
    df_trades.index = pd.to_datetime(df_trades.index)
    df_trades = df_trades.mul(rv,axis='columns').dropna().fillna(0)
    df_diff = pd.DataFrame(index=df_trades.index)
    df_diff['impact']    = (abs(df_trades)*impact).sum(axis=1)
    df_diff['commission'] = (df_trades.iloc[:,:]!=0).sum(axis=1)*commission
    
    df_trades['Cash'] = -1*df_trades.sum(axis=1) - df_diff['commission'] - df_diff['impact']

    df_holdings = df_holdings.cumsum(axis=0)


    df_holdings['Cash'] =  start_val + df_trades['Cash'].cumsum(axis=0)

    temp           = df_holdings.iloc[:,:-1].copy()
    df_value       = temp.mul(df_prices,axis='columns')
    df_value['Value'] = df_value.sum(axis=1)

    df_portval = pd.DataFrame(index=range(len(df_value.index.get_values())))
    #pdb.set_trace()
    df_portval['TotalValue']=0
    posicion = 0
    for key,value in df_value.iterrows():
        df_portval.ix[posicion,'TotalValue'] = df_value.ix[posicion,'Value'] + df_holdings.ix[posicion,'Cash']
        posicion = posicion + 1

    df_portval['Dates'] = df_value.index.get_values()
    
    df_portval = df_portval.drop_duplicates(subset='Dates',keep='last')
   
    df_portval = pd.DataFrame(data=df_portval.TotalValue.values,index=df_portval.Dates.values,columns=['TotalValue'])
    
    
    #pd.concat([df_value['Value'],df_holdings['Cash'],df_portval,orders_df['Order']],axis=1)
    return df_portval




In [0]:

import pandas as pd
import numpy as np
import datetime as dt
#from util import get_data, plot_data
import matplotlib.pyplot as plt
import pdb

# This is the function that will be tested by the autograder
# The student must update this code to properly implement the functionality


def daily_returns(df):
    df1 = df.copy()
    df2 = (df1[1:]/df1[:-1].values) - 1
    return(df2)

def compound_returns(df):
    df1 = df.copy()
    df1 = df1/df1.iloc[0] - 1
    df1.iloc[0,] = 0 
    return(df1)
def normalize_data(df):
    df1 = df.copy()
    return (df1/df1.iloc[0])
    
def plot_data(df,title='Normalized Data'):
    ax=df.plot(title=title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.grid(linestyle='--', linewidth=1)
    plt.show()
    


def assess_portfolio(portfolio=None,sd = dt.datetime(2008,1,1), ed = dt.datetime(2009,1,1), \
    syms = ['GOOG','AAPL','GLD','XOM'], \
    allocs=[0.1,0.2,0.3,0.4], \
    sv=1000000, rfr=0.0, sf=252.0,precios=0, \
    gen_plot=False):

    # Read in adjusted closing prices for given symbols, date range
    dates = pd.date_range(sd, ed)
    if(type(portfolio)==type(None)):
        prices_all = get_data(syms, dates)  # automatically adds SPY
        prices = prices_all[syms]  # only portfolio symbols
        prices_SPY = prices_all['SPY']  # only SPY, for comparison later
    else:
        #pdb.set_trace()
        prices_all = get_data(syms,dates)
        prices = portfolio# pd.concat([portfolio,prices_all[syms]],axis=1)
        prices_SPY = prices_all[syms]
    # Get daily portfolio value

    
    
    
    if( (isinstance(precios,pd.core.series.Series))):
        pdb.set_trace()
        precios = precios.to_frame()
        normalized_values = normalize_data(precios)
        
    else:
        normalized_values = normalize_data(prices)

    
    alloced_values    = normalized_values*allocs
    pos_val           = alloced_values*sv
    port_val          = pos_val.sum(axis=1) # add code here to compute daily portfolio values

    d_returns         = daily_returns(port_val)
    cr                = port_val.iloc[-1]/port_val.iloc[0] -1
    adr               = d_returns.mean()
    sddr              = d_returns.std()
    sr                = (adr/sddr)*np.sqrt(252)
    prices_SPY = normalize_data(prices_SPY)
    port_val   = normalize_data(port_val)
    port_val   = pd.DataFrame(data=port_val,index=port_val.index,columns=['Portfolio'])
    # Get portfolio statistics (note: std_daily_ret = volatility)
    #cr, adr, sddr, sr = [0.25, 0.001, 0.0005, 2.1] # add code here to compute stats

    # Compare daily portfolio value with SPY using a normalized plot
    if gen_plot:
        # add code to plot here
        if(type(portfolio)==type(None)):
            df_temp = pd.concat([port_val, prices_SPY], keys=['Portfolio', 'SPY'], axis=1)
        else:
            #pdb.set_trace()
            df_temp = pd.concat([port_val, prices_SPY], axis=1)
        plot_data(df_temp)

    # Add code here to properly compute end value
    ev = sv

    return(cr, adr, sddr, sr, ev)








In [0]:
"""
Template for implementing StrategyLearner  (c) 2016 Tucker Balch
"""

import datetime as dt
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import pdb
import pandas as pd
import time
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


class StrategyLearner(object):   
    def __init__(self, verbose=False, impact=0.005 , commission=9.95):
        self.verbose = verbose
        self.impact = impact
        self.commission = commission
 

    def calculate_reward(self, holdings_actions_1 , action ,key):

        prices_all = self.prices_all
        

        holdings_actions              = {0:1,1:0.5,2:0,3:-0.5,4:-1}[action]
        
                
        if( key == prices_all.index[0]):
            
            holdings_diff             = 0
            
        else:
            price_t                   = prices_all.iloc[prices_all.index.get_loc(key)-1]
            price_t_plus_1            = prices_all.loc[key]
            cash                      = -1*(holdings_actions-holdings_actions_1) * price_t
            holdings_diff             = holdings_actions * price_t_plus_1 - holdings_actions_1 * price_t + cash
            
        if(holdings_actions_1 - holdings_actions)!=0:
            
            holdings_diff     = holdings_diff - self.commission
            
        reward = holdings_diff
        
        
        return (reward,holdings_actions)
                
    def addEvidence(self, symbol = "IBM", \
        sd=dt.datetime(2015,1,1), \
        ed=dt.datetime(2017,1,1), \
        sv = 1000000): 
        # add your code to do learning here
        
        btc = pd.read_csv('https://www.dropbox.com/s/rmh9rramnsya81h/COINBASE_FILTERED.CSV?dl=1',sep=',')
        
        size= int(len(btc)*0.005)
        
        btc = btc.iloc[-3*size:-size]
        
        btc[btc.columns.values] = btc[btc.columns.values].ffill()
        
        btc['TR'] =  0 
        
        a = btc['High']-btc['Low']
        b = btc['Low']-btc['Close'].shift(-1)
        c = btc['High']-btc['Close'].shift(-1)
        
        btc['TR'] = pd.concat([a,b,c],axis=1).max(axis=1)
        btc['ATR'] = btc['TR'].ewm(span = 10).mean()
        
        btc['Delta'] = btc['Close'] - btc['Open']
        
        btc['to_predict'] = btc['Delta'].apply(lambda x : 1 if(x>0) else 0)
        
        btc.index = pd.to_datetime(btc['Timestamp'],infer_datetime_format =True,unit='s')
        
        (normalized_values ,bbp,moving_avarage,rsi_val,momentum,sma_cross) = indicators(data=btc)
        
        norm_val          = normalized_values.copy()
                
        states            = pd.concat([normalized_values,bbp,moving_avarage,rsi_val,momentum,sma_cross],axis=1).apply(lambda x : x.fillna(0)).iloc[13:,:]
                
        self.prices       = normalized_values.copy()

        state_size        = 6
        action_size       = 5
        
        max_iter          = 10
        actions_df        = pd.DataFrame(index=states.index,data=[0]*len(states))
        iter_num          = 0
        converged         = False
        
        dates = pd.date_range(sd,ed)
        self.prices_all   = btc['Weighted_Price'] #ut.get_data([symbol], dates)[symbol] 
                
        agent = DQNAgent(state_size, action_size)
        
        batch_size = 2 #64/32
        
        comienzo = time.time()

        for e in range(max_iter):
            
            print("Va por la iteracion ", e)
            
            holdings_actions = 0
            syms             = [symbol]
            X                = np.array([states.iloc[0]])
            action           = 2

            
            for key,row in states.iloc[1:].iterrows():
                
                                
                (reward,holdings_actions) = self.calculate_reward( holdings_actions_1 = holdings_actions , action = action ,key= key)
                
                next_state = np.array([row])
                
                agent.remember(X, action, reward, next_state)
                
                
                if len(agent.memory) > batch_size:
                    agent.replay(batch_size)
                    X = np.array([row])
                
                
                holdings_actions_1 = holdings_actions
                
                action = agent.act(X)
                
                actions_df.loc[key,iter_num] = holdings_actions

            iter_num += 1
            
            
            converged=False    
        print(time.time()-comienzo)
        previous_days = 13
        trades = pd.DataFrame(data = actions_df.iloc[:,-1],index = actions_df.index).diff().shift(-1).fillna(0)
        
        trades.sort_index(axis =0 , inplace=True)
        trades.iloc[-1] = -1*trades.iloc[:-1].sum()
        
        trades.columns = ['Shares']
        
        trades['Symbol'] = symbol
        trades['Order']  = trades['Shares'].to_frame().applymap(lambda x : {-2:'SELL',-1.5:'SELL',-1:'SELL',-0.5:'SELL',0:0,0.5:"BUY",1:"BUY",1.5:"BUY",2:"BUY"}[x])
        trades['Shares'] = trades['Shares'].abs()
        trades['Date']   = trades.index
        compute_portvals(trades)
        pdb.set_trace()
        self.agent = agent
        
        
    def testPolicy(self, symbol = "IBM", \
        sd=dt.datetime(2009,1,1), \
        ed=dt.datetime(2010,1,1), \
        sv = 10000):
        
        btc = pd.read_csv('https://www.dropbox.com/s/rmh9rramnsya81h/COINBASE_FILTERED.CSV?dl=1',sep=',')
        
        size= int(len(btc)*0.005)
        btc = btc.iloc[-size:]
        
        btc[btc.columns.values] = btc[btc.columns.values].ffill()
        
        btc['TR'] =  0 
        
        a = btc['High']-btc['Low']
        b = btc['Low']-btc['Close'].shift(-1)
        c = btc['High']-btc['Close'].shift(-1)
        
        btc['TR'] = pd.concat([a,b,c],axis=1).max(axis=1)
        btc['ATR'] = btc['TR'].ewm(span = 10).mean()
        
        btc['Delta'] = btc['Close'] - btc['Open']
        
        btc['to_predict'] = btc['Delta'].apply(lambda x : 1 if(x>0) else 0)
        
        btc.index = pd.to_datetime(btc['Timestamp'],infer_datetime_format =True,unit='s')
        
        self.prices_all   = btc['Weighted_Price'] #ut.get_data([symbol], dates)[symbol]
        
        (normalized_values ,bbp,moving_avarage,rsi_val,momentum,sma_cross) = indicators(data=btc)
                
                
        norm_val          = normalized_values.copy()
        
        states            = pd.concat([normalized_values,bbp,moving_avarage,rsi_val,momentum,sma_cross],axis=1).apply(lambda x : x.fillna(0)).iloc[13:,:]
        
        actions_df        = pd.DataFrame(index=states.index,data=[0]*len(states))
        
        holdings          = pd.DataFrame(data = [0]*len(states) , index= states.index)
        
        for key,state in states.iterrows():
            action = self.agent.act(np.array([state]))

            holdings.loc[key] = {0:-1,1:-0.5,2:0 ,3:0.5 , 4:1}[action]

        pdb.set_trace()
        
        
        previous_days = 13
        trades = pd.DataFrame(data = actions_df.iloc[:,-1],index = actions_df.index).diff().shift(-1).fillna(0)
        
        trades.sort_index(axis =0 , inplace=True)
        trades.iloc[-1] = -1*trades.iloc[:-1].sum()
        
        
        
        trades.columns = ['Shares']
        

        trades['Symbol'] = symbol
        trades['Order']  = trades['Shares'].to_frame().applymap(lambda x : {-2:'SELL',-1.5:'SELL',-1:'SELL',-0.5:'SELL',0:0,0.5:"BUY",1:"BUY",1.5:"BUY",2:"BUY"}[x])
        trades['Shares'] = trades['Shares'].abs()
        trades['Date']   = trades.index
        self.agent       = agent
        
        if self.verbose: print(prices_all)
        return trades

if __name__=="__main__":
       
    nuevo = StrategyLearner(commission=9.95)
    nuevo.addEvidence(symbol="JPM")
    nuevo.testPolicy(symbol="JPM",sv=1000000)
    print ("One does not simply think up a strategy")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: 
.ix is deprecated. Please use
.loc for la

Va por la iteracion  0
Va por la iteracion  1
